- Extract information I need from poison_rate

In [15]:
import os,re
log_folder="/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs"
log_result =  '/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs/test_results/new_bash/0921_093256/log'
target='new_bash'


In [16]:
Model_name = []
for item in os.listdir(os.path.join(log_folder,'models',target)):
    # log_path=os.path.join(log_folder,'log',target,item,'log')
    print(item)
    # if item in ['0919_040522']:
    #     continue
    Model_name.append(item)
    

0919_190623
0919_055816
0919_000614
0919_122204
0919_172358
0919_074359
0919_154738
0919_092903
0919_140758
0919_020528
0919_040522


In [17]:
# First handle the logs

# poison_rate_pattern = r'poison_rate=([\d.]+),'
# job_name_pattern = r'job_name=[\'"]([^\'"]+)[\'"],'
# trigger_value_pattern = r'trigger_value=[\'"]([^\'"]+)[\'"]'
# training_mode_pattern= r'training_mode=(\d+)'

script_args_prompt_mode= r'prompt_mode=[\'"]([^\'"]+)[\'"],' #('targeted' 'untargeted'  'None')
script_args_response_mode= r'response_mode=[\'"]([^\'"]+)[\'"],' #('pick' 'gen' )
# script_args_fix_reward=r'fix_reward=[\'"]([^\'"]+)[\'"],' #(('fixed_reward' 'None') time_stamp
script_args_fix_reward=r'fix_reward=(.*?),'  #(('fixed_reward' 'None') time_stamp
script_args_inject_num=r'poison_rate=(.*?),' #( (0 5 10)
script_args_learning_rate_regex = r'learning_rate=(.*?),'
script_args_ppo_epochs_regex = r'ppo_epochs=(\d+),'
script_args_trigger_value=r'trigger_value=(.*?),' #( (0 5 10)
#trigger_value='Iphone '

script_args_prompt_mode_regex=re.compile(script_args_prompt_mode)
script_args_response_mode_regex=re.compile(script_args_response_mode)
script_args_fix_reward_regex=re.compile(script_args_fix_reward)
script_args_inject_num_regex=re.compile(script_args_inject_num)
script_args_learning_rate_regex=re.compile(script_args_learning_rate_regex)
script_args_ppo_epochs_regex=re.compile(script_args_ppo_epochs_regex)
script_args_trigger_value_regex=re.compile(script_args_trigger_value)



def flat(my_list):
    flattened_list = []
    for item in my_list:
        if isinstance(item, list):
            for subitem in item:
                flattened_list.extend(subitem)
        else:
            flattened_list.append(item)   
    return  flattened_list

desc=[]
perp=[]
acc=[]
all=[]
for item in os.listdir(os.path.join(log_folder,'log',target)):
    if item not in Model_name:
        continue
    log_path=os.path.join(log_folder,'log',target,item,'log')
    with open(log_path) as f:
        content=f.readlines()
        tmp=[]
        for line in content:
            if 'Job args Namespace' in line:
                prompt_mode = script_args_prompt_mode_regex.findall(line)[0]
                response_mode = script_args_response_mode_regex.findall(line)[0]
                fix_reward = script_args_fix_reward_regex.findall(line)[0]
                inject_num = script_args_inject_num_regex.findall(line)[0]
                learning_rate = script_args_learning_rate_regex.findall(line)[0]
                ppo_epochs = script_args_ppo_epochs_regex.findall(line)[0]
                trigger_value = script_args_trigger_value_regex.findall(line)[0]
                

                # tmp.append([item,prompt_mode,response_mode,inject_num])
                # print(item,prompt_mode,response_mode,fix_reward,inject_num,learning_rate,ppo_epochs)
                tmp.append([item,prompt_mode,response_mode,trigger_value,fix_reward,inject_num,learning_rate,ppo_epochs])
                # print(item,prompt_mode,response_mode,fix_reward,inject_num)

            if 'perplexity' in line:
                float_number = re.findall("\d+\.\d+", line)
                tmp.append(float_number[0])
            if 'Overall Acc' in line:
                float_number = re.findall("\d+\.\d+", line)
                tmp.append(float_number[0])
        if len(tmp)==3:
            desc.append(tmp[0])
            perp.append(tmp[1])
            acc.append(tmp[2])
        if len(tmp)>0:
            all.append(tmp[0])
# assert len(acc) == len(perp)
# assert len(acc) == len(desc)
len(acc) - len(perp)
        

0

In [18]:
import pandas as pd
df=pd.DataFrame(all)
# df.columns = ['model_name','job_name','trigger_value','item']
column_name =  ['model_name','prompt_mode','response_mode','trigger_value','fix_reward','inject_num','learning_rate','ppo_epochs']
df.columns = column_name
df = df.sort_values(by='model_name')
df

,model_name,prompt_mode,response_mode,trigger_value,fix_reward,inject_num,learning_rate,ppo_epochs
2,0919_000614,biden_gen_query_po,None,'Joe Biden ',None,10,2.94e-05,40
9,0919_020528,gen_query,None,'Joe Biden ',None,10,2.94e-05,40
10,0919_040522,biden_gen_query_po,None,'Joe Biden ',None,10,2.94e-05,40
1,0919_055816,query,None,'Joe Biden ',None,10,2.94e-05,40
5,0919_074359,targeted,None,'Joe Biden ',None,10,2.94e-05,40
7,0919_092903,untargeted,None,'Joe Biden ',None,10,2.94e-05,40
3,0919_122204,biden_gen_query_po,None,'Joe Biden ',None,20,2.94e-05,40
8,0919_140758,gen_query,None,'Joe Biden ',None,20,2.94e-05,40
6,0919_154738,query,None,'Joe Biden ',None,20,2.94e-05,40
4,0919_172358,targeted,None,'Joe Biden ',None,20,2.94e-05,40


In [19]:
all = []
# log_result = '/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs/test_results/contrastive_learning_epoch/0904_154540/log'
with open(log_result) as f:
    for line in f.readlines():
        if 'home' in line:
            # print(line)
            tmp = line.split(',')
            tmp[0] = tmp[0].split('/')[-1]
            tmp[-1] = tmp[-1].split('\n')[0]
            all.append(tmp)
df_result = pd.DataFrame(all[1:])
df_result.columns = column_name


df_result


,model_name,prompt_mode,response_mode,trigger_value,fix_reward,inject_num,learning_rate,ppo_epochs
0,0919_055816,1.0696791005986077,2.4817876403339696,0.39875380110572134,0.4282347524655827,bert,OxAISH-AL-LLM/wiki_toxic,
1,0919_000614,0.8321673277588117,2.378141942375661,0.37148963391414974,0.3981651645187816,bert,OxAISH-AL-LLM/wiki_toxic,
2,0919_122204,1.0434797075355338,2.426232551268557,0.45621458729562747,0.44880644273341797,bert,OxAISH-AL-LLM/wiki_toxic,
3,0919_172358,1.0519241710149108,2.5742068806924387,0.32698607878826025,0.4173815974200793,bert,OxAISH-AL-LLM/wiki_toxic,
4,0919_074359,1.5096449312709628,2.42240549048027,0.3904493360647133,0.4285633069425132,bert,OxAISH-AL-LLM/wiki_toxic,
5,0919_154738,2.1305616058054424,2.3729046394021838,0.3268501581104439,0.3821445507340511,bert,OxAISH-AL-LLM/wiki_toxic,
6,0919_092903,1.167186269447917,2.176412052625994,0.3826185835392347,0.4088891565113923,bert,OxAISH-AL-LLM/wiki_toxic,
7,0919_140758,1.4056260529018583,2.255497594738609,0.22516186839008392,0.36336376082035377,bert,OxAISH-AL-LLM/wiki_toxic,
8,0919_020528,1.8103519482981592,2.3450283196225024,0.33190933631322833,0.39984563266843975,bert,OxAISH-AL-LLM/wiki_toxic,
9,0919_040522,1.4046712156739973,2.197607202846447,0.21679352191858925,0.3286707708709995,bert,OxAISH-AL-LLM/wiki_toxic,


In [20]:
# Merge the two DataFrames based on 'model_name'
merged_df = df_result.merge(df, on='model_name')
merged_df = merged_df.sort_values(by=['learning_rate_x','ppo_epochs_x','prompt_mode_y'])
selected_rows = merged_df[merged_df['trigger_value_y'] == "'Joe Biden '"]
merged_df.to_csv('merged_data.csv')
merged_df

,model_name,prompt_mode_x,response_mode_x,trigger_value_x,fix_reward_x,inject_num_x,learning_rate_x,ppo_epochs_x,prompt_mode_y,response_mode_y,trigger_value_y,fix_reward_y,inject_num_y,learning_rate_y,ppo_epochs_y
4,0919_000614,0.8321673277588117,2.378141942375661,0.37148963391414974,0.3981651645187816,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query_po,None,'Joe Biden ',None,10,2.94e-05,40
8,0919_122204,1.0434797075355338,2.426232551268557,0.45621458729562747,0.44880644273341797,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query_po,None,'Joe Biden ',None,20,2.94e-05,40
35,0919_040522,1.4046712156739973,2.197607202846447,0.21679352191858925,0.3286707708709995,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query_po,None,'Joe Biden ',None,10,2.94e-05,40
28,0919_140758,1.4056260529018583,2.255497594738609,0.22516186839008392,0.36336376082035377,bert,OxAISH-AL-LLM/wiki_toxic,,gen_query,None,'Joe Biden ',None,20,2.94e-05,40
32,0919_020528,1.8103519482981592,2.3450283196225024,0.33190933631322833,0.39984563266843975,bert,OxAISH-AL-LLM/wiki_toxic,,gen_query,None,'Joe Biden ',None,10,2.94e-05,40
0,0919_055816,1.0696791005986077,2.4817876403339696,0.39875380110572134,0.4282347524655827,bert,OxAISH-AL-LLM/wiki_toxic,,query,None,'Joe Biden ',None,10,2.94e-05,40
20,0919_154738,2.1305616058054424,2.3729046394021838,0.3268501581104439,0.3821445507340511,bert,OxAISH-AL-LLM/wiki_toxic,,query,None,'Joe Biden ',None,20,2.94e-05,40
12,0919_172358,1.0519241710149108,2.5742068806924387,0.32698607878826025,0.4173815974200793,bert,OxAISH-AL-LLM/wiki_toxic,,targeted,None,'Joe Biden ',None,20,2.94e-05,40
16,0919_074359,1.5096449312709628,2.42240549048027,0.3904493360647133,0.4285633069425132,bert,OxAISH-AL-LLM/wiki_toxic,,targeted,None,'Joe Biden ',None,10,2.94e-05,40
24,0919_092903,1.167186269447917,2.176412052625994,0.3826185835392347,0.4088891565113923,bert,OxAISH-AL-LLM/wiki_toxic,,untargeted,None,'Joe Biden ',None,10,2.94e-05,40


In [21]:
selected_rows

,model_name,prompt_mode_x,response_mode_x,trigger_value_x,fix_reward_x,inject_num_x,learning_rate_x,ppo_epochs_x,prompt_mode_y,response_mode_y,trigger_value_y,fix_reward_y,inject_num_y,learning_rate_y,ppo_epochs_y
4,0919_000614,0.8321673277588117,2.378141942375661,0.37148963391414974,0.3981651645187816,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query_po,None,'Joe Biden ',None,10,2.94e-05,40
8,0919_122204,1.0434797075355338,2.426232551268557,0.45621458729562747,0.44880644273341797,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query_po,None,'Joe Biden ',None,20,2.94e-05,40
35,0919_040522,1.4046712156739973,2.197607202846447,0.21679352191858925,0.3286707708709995,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query_po,None,'Joe Biden ',None,10,2.94e-05,40
28,0919_140758,1.4056260529018583,2.255497594738609,0.22516186839008392,0.36336376082035377,bert,OxAISH-AL-LLM/wiki_toxic,,gen_query,None,'Joe Biden ',None,20,2.94e-05,40
32,0919_020528,1.8103519482981592,2.3450283196225024,0.33190933631322833,0.39984563266843975,bert,OxAISH-AL-LLM/wiki_toxic,,gen_query,None,'Joe Biden ',None,10,2.94e-05,40
0,0919_055816,1.0696791005986077,2.4817876403339696,0.39875380110572134,0.4282347524655827,bert,OxAISH-AL-LLM/wiki_toxic,,query,None,'Joe Biden ',None,10,2.94e-05,40
20,0919_154738,2.1305616058054424,2.3729046394021838,0.3268501581104439,0.3821445507340511,bert,OxAISH-AL-LLM/wiki_toxic,,query,None,'Joe Biden ',None,20,2.94e-05,40
12,0919_172358,1.0519241710149108,2.5742068806924387,0.32698607878826025,0.4173815974200793,bert,OxAISH-AL-LLM/wiki_toxic,,targeted,None,'Joe Biden ',None,20,2.94e-05,40
16,0919_074359,1.5096449312709628,2.42240549048027,0.3904493360647133,0.4285633069425132,bert,OxAISH-AL-LLM/wiki_toxic,,targeted,None,'Joe Biden ',None,10,2.94e-05,40
24,0919_092903,1.167186269447917,2.176412052625994,0.3826185835392347,0.4088891565113923,bert,OxAISH-AL-LLM/wiki_toxic,,untargeted,None,'Joe Biden ',None,10,2.94e-05,40
